In [5]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.5. Cluster Distribution Analyses/1.5.1. Wilcoxon Rank Sum/Output')
cur_date = "052021"

library(readxl)
library(data.table)
library(ggplot2)
library(factoextra)
library(janitor)
library(dplyr)
library(tidyverse)
library(gridExtra)
library(cluster)
library(vegan)
library(fpc)
library(ggdendro)

#reading in file
cytokines <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))
subjects = data.frame(read_excel("SubjectInfo_102920.xlsx", sheet = 2))

#cluster assignments for each compartment 
NELF_clus <- data.frame(read_excel("021421NELF_cluster_assignments.xlsx")) 
NLF_clus <- data.frame(read_excel("021421NLF_cluster_assignments.xlsx"))
Serum_clus <- data.frame(read_excel("021421Serum_cluster_assignments.xlsx"))
Sputum_clus <- data.frame(read_excel("021421Sputum_cluster_assignments.xlsx"))

Assigning the same baseline clusters to e-cig and cigarette smokers and running wilcoxon rank sum tests to see if they differ. 

# Eigencytokines

In [6]:
get_eigencytokines = function(smoking_status){
    # Separating the cytokine data into compartment dfs
    cytokines <- cytokines %>% 
        filter(Group == smoking_status) %>% #only want non-smokers for baseline analysis
        group_by(Compartment) %>% 
        group_split
    NELF <- cytokines[[1]]
    NLF <- cytokines[[2]]
    Serum <- cytokines[[3]]
    Sputum <- cytokines[[4]]

    # reshaping data 
    NELF <- reshape2::dcast(NELF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    NLF <- reshape2::dcast(NLF, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Serum <- reshape2::dcast(Serum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 
    Sputum <- reshape2::dcast(Sputum, SubjectID ~ Protein, value.var="Conc_pslog2") %>% 
      column_to_rownames("SubjectID") 

    #background filter eliminating any cytokines that are not expressed in a compartment  
    NLF$I309 <- NULL
    Sputum$I309 <- NULL 

    # the scale function operates across columns 
    NELF_scaled <- NELF %>% 
      scale() %>% 
      as.data.frame()
    NLF_scaled <- NLF %>% 
      scale() %>% 
      as.data.frame()
    Serum_scaled <- Serum %>% 
      scale() %>% 
      as.data.frame()
    Sputum_scaled <- Sputum %>% 
      scale() %>% 
      as.data.frame()

    #transpose cytokine data for each compartment
    NELF <- as.data.frame(t(NELF))
    NLF <- as.data.frame(t(NLF)) 
    Serum <- as.data.frame(t(Serum))
    Sputum <- as.data.frame(t(Sputum)) 

    #renaming first column, grouping and splitting by "Cluster" column
    NELF_clus <- NELF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    NLF_clus <- NLF_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Serum_clus <- Serum_clus %>% 
      group_by(Cluster) %>% 
      group_split
    Sputum_clus <- Sputum_clus %>% 
      group_by(Cluster) %>% 
      group_split

    #making dfs for each cluster for PCA analysis 
    NELF_1 <- NELF_clus[[1]]
    NELF_2 <- NELF_clus[[2]]
    NELF_3 <- NELF_clus[[3]]

    NLF_1 <- NLF_clus[[1]]
    NLF_2 <- NLF_clus[[2]]
    NLF_3 <- NLF_clus[[3]]

    Serum_1 <- Serum_clus[[1]]
    Serum_2 <- Serum_clus[[2]]
    Serum_3 <- Serum_clus[[3]]

    Sputum_1 <- Sputum_clus[[1]]
    Sputum_2 <- Sputum_clus[[2]]
    Sputum_3 <- Sputum_clus[[3]]

    #making df with subjects' cytokine concentration data for each cluster 
    NELF_1 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_2 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NELF_3 <- NELF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NELF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    NLF_1 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_2 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    NLF_3 <- NLF %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% NLF_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Serum_1 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_2 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Serum_3 <- Serum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Serum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    Sputum_1 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_1$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_2 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_2$Cytokine) %>% 
      column_to_rownames(var="Cytokine")
    Sputum_3 <- Sputum %>% 
      rownames_to_column("Cytokine") %>% 
      filter(Cytokine %in% Sputum_3$Cytokine) %>% 
      column_to_rownames(var="Cytokine")

    #PCA on each cluster, eigenvectors are in rotation -- PROBLEM - for some reason had to convert everything to numeric  
    pca_NELF_1 <- NELF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_2 <- NELF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NELF_3 <- NELF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>%   
      prcomp(center = TRUE, scale = TRUE)

    pca_NLF_1 <- NLF_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_2 <- NLF_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_NLF_3 <- NLF_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    pca_Serum_1 <- Serum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Serum_2 <- Serum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Serum_3 <- Serum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_1 <- Sputum_1 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_2 <- Sputum_2 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)
    pca_Sputum_3 <- Sputum_3 %>% 
      lapply(as.numeric) %>% 
      as.data.frame() %>% 
      prcomp(center = TRUE, scale = TRUE)

    #eigenvector dfs of first principal component 
    eigencytokines_NELF_1 <- data.frame(pca_NELF_1$rotation[,"PC1"])
      colnames(eigencytokines_NELF_1)[1] <- "ClusterA"
    eigencytokines_NELF_2 <- data.frame(pca_NELF_2$rotation[,"PC1"])
      colnames(eigencytokines_NELF_2)[1] <- "ClusterB"
    eigencytokines_NELF_3 <- data.frame(pca_NELF_3$rotation[,"PC1"])
      colnames(eigencytokines_NELF_3)[1] <- "ClusterC"

    eigencytokines_NLF_1 <- data.frame(pca_NLF_1$rotation[,"PC1"])
      colnames(eigencytokines_NLF_1)[1] <- "ClusterA"
    eigencytokines_NLF_2 <- data.frame(pca_NLF_2$rotation[,"PC1"])
      colnames(eigencytokines_NLF_2)[1] <- "ClusterB"
    eigencytokines_NLF_3 <- data.frame(pca_NLF_3$rotation[,"PC1"])
      colnames(eigencytokines_NLF_3)[1] <- "ClusterC"

    eigencytokines_Serum_1 <- data.frame(pca_Serum_1$rotation[,"PC1"])
      colnames(eigencytokines_Serum_1)[1] <- "ClusterA"
    eigencytokines_Serum_2 <- data.frame(pca_Serum_2$rotation[,"PC1"])
      colnames(eigencytokines_Serum_2)[1] <- "ClusterB"
    eigencytokines_Serum_3 <- data.frame(pca_Serum_3$rotation[,"PC1"])
      colnames(eigencytokines_Serum_3)[1] <- "ClusterC"

    eigencytokines_Sputum_1 <- data.frame(pca_Sputum_1$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_1)[1] <- "ClusterA"
    eigencytokines_Sputum_2 <- data.frame(pca_Sputum_2$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_2)[1] <- "ClusterB"
    eigencytokines_Sputum_3 <- data.frame(pca_Sputum_3$rotation[,"PC1"])
      colnames(eigencytokines_Sputum_3)[1] <- "ClusterC"

    #collapse all eigencytokine dfs
    eigencytokines_NELF <- cbind(eigencytokines_NELF_1, eigencytokines_NELF_2, eigencytokines_NELF_3)
    eigencytokines_NLF <- cbind(eigencytokines_NLF_1, eigencytokines_NLF_2, eigencytokines_NLF_3)
    eigencytokines_Serum <- cbind(eigencytokines_Serum_1, eigencytokines_Serum_2, eigencytokines_Serum_3)
    eigencytokines_Sputum <- cbind(eigencytokines_Sputum_1, eigencytokines_Sputum_2, eigencytokines_Sputum_3)
    
    all_eigencytokines = cbind(eigencytokines_NLF, eigencytokines_NELF, eigencytokines_Sputum, eigencytokines_Serum)
    return(all_eigencytokines)
}

#calling fn
eigencytokines_NLF_NS = get_eigencytokines("NS")[1:3]
eigencytokines_NELF_NS = get_eigencytokines("NS")[4:6]
eigencytokines_Sputum_NS = get_eigencytokines("NS")[7:9]
eigencytokines_Serum_NS = get_eigencytokines("NS")[10:12]
eigencytokines_NLF_Ecig = get_eigencytokines("Ecig")[1:3]
eigencytokines_NELF_Ecig = get_eigencytokines("Ecig")[4:6]
eigencytokines_Sputum_Ecig = get_eigencytokines("Ecig")[7:9]
eigencytokines_Serum_Ecig = get_eigencytokines("Ecig")[10:12]
eigencytokines_NLF_CS = get_eigencytokines("CS")[1:3]
eigencytokines_NELF_CS = get_eigencytokines("CS")[4:6]
eigencytokines_Sputum_CS = get_eigencytokines("CS")[7:9]
eigencytokines_Serum_CS = get_eigencytokines("CS")[10:12]

In [7]:
#exporting eigencytokines these are different than the eigencytokines from the baseline analysis, because these
#now include CS and Ecig users
eigencytokines_NLF = rbind(eigencytokines_NLF_NS, eigencytokines_NLF_CS, eigencytokines_NLF_Ecig)
eigencytokines_NELF = rbind(eigencytokines_NELF_NS, eigencytokines_NELF_CS, eigencytokines_NELF_Ecig)
eigencytokines_Sputum = rbind(eigencytokines_Sputum_NS, eigencytokines_Sputum_CS, eigencytokines_Sputum_Ecig)
eigencytokines_Serum = rbind(eigencytokines_Serum_NS, eigencytokines_Serum_CS, eigencytokines_Serum_Ecig)

# write.csv(eigencytokines_NLF, paste0(Output,"/", cur_date, "_NLF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_NELF, paste0(Output,"/", cur_date, "_NELF_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_Sputum, paste0(Output,"/", cur_date, "_Sputum_eigencytokines.csv"), row.names = TRUE)
# write.csv(eigencytokines_Serum, paste0(Output,"/", cur_date, "_Serum_eigencytokines.csv"), row.names = TRUE)

# Wilcoxon Rank Sum tests

Comparing eigencytokines of non-smokers to smokers. 

In [8]:
#converting subject ids to col, melting, and adding compartment
changed_df = function(df, compartment_name){
    df = reshape2::melt(df %>%
        rownames_to_column(var = "SubjectID"), variable = "Cluster",  value.name = 'Conc_pslog2')
    df$Compartment = rep(compartment_name, times = length(df$SubjectID))
    return(df)
}
NS_eigencytokines_NLF = changed_df(eigencytokines_NLF_NS,"NLF")
NS_eigencytokines_NELF = changed_df(eigencytokines_NELF_NS,"NELF")
NS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_NS,"Sputum")
NS_eigencytokines_Serum = changed_df(eigencytokines_Serum_NS,"Serum")
Ecig_eigencytokines_NLF = changed_df(eigencytokines_NLF_Ecig,"NLF")
Ecig_eigencytokines_NELF = changed_df(eigencytokines_NELF_Ecig,"NELF")
Ecig_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_Ecig,"Sputum")
Ecig_eigencytokines_Serum = changed_df(eigencytokines_Serum_Ecig,"Serum")
CS_eigencytokines_NLF = changed_df(eigencytokines_NLF_CS,"NLF")
CS_eigencytokines_NELF = changed_df(eigencytokines_NELF_CS,"NELF")
CS_eigencytokines_Sputum = changed_df(eigencytokines_Sputum_CS,"Sputum")
CS_eigencytokines_Serum = changed_df(eigencytokines_Serum_CS,"Serum")

#recombining into 3 dataframes
NS_eigencytokines = rbind(NS_eigencytokines_NLF, NS_eigencytokines_NELF, NS_eigencytokines_Sputum, 
                          NS_eigencytokines_Serum)
CS_eigencytokines = rbind(CS_eigencytokines_NLF, CS_eigencytokines_NELF, CS_eigencytokines_Sputum, 
                          CS_eigencytokines_Serum)
Ecig_eigencytokines = rbind(Ecig_eigencytokines_NLF, Ecig_eigencytokines_NELF, Ecig_eigencytokines_Sputum, 
                          Ecig_eigencytokines_Serum)

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables

Using SubjectID as id variables



In [9]:
#creating vectors to loop through
compartment = c('NLF','NELF','Sputum','Serum')
cluster = unique(NS_eigencytokines_NLF$Cluster)

#initializing vectors to store values
CS_df = data.frame()
Ecig_df = data.frame()
wilcoxon_rank_sum_values = function(df1, df2, empty_df){
    #running wilcoxon rank sum and storing the statistic, compartment, cluster, and p value in a vector
    for (i in 1:length(compartment)){
        for (j in 1: length(cluster)){
            variable1_df = df1 %>% # baseline df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)
            variable2_df = df2 %>% # smoker df
                filter(Compartment == compartment[i], Cluster == cluster[j]) %>%
                select(Compartment, Cluster, Conc_pslog2)


            #running wilcoxon rank sum
            wilcox_test = wilcox.test(variable1_df$Conc_pslog2, variable2_df$Conc_pslog2)
            
            #calculating absolute difference
            AD = (mean(variable2_df$Conc_pslog2) - mean(variable1_df$Conc_pslog2))

            #contains compartment, cluster, u stat, p value
            values_df = cbind(compartment[i], as.character(cluster[j]), AD, wilcox_test$statistic, wilcox_test$p.value)
            empty_df = rbind(empty_df, values_df)

        }
    }
    colnames(empty_df) = c("Compartment",'Cluster', 'AD','Stat', 'P Value')
    return(empty_df)
}

#calling fn
CS_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, CS_eigencytokines, CS_df)
Ecig_wilcoxon_values = wilcoxon_rank_sum_values(NS_eigencytokines, Ecig_eigencytokines, Ecig_df)

In [13]:
final_table = function(df){
    #"""
    #Adding a padj col
    
    #:param: vector, demographic variable
    #:output: a 6x12 df containing compartment, cluster, protein, u stat, p value, p adj

    #"""
    
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = df %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(as.numeric(as.character(single_compartment_df$`P Value`)), method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    df = cbind(df, PAdj)
    return(df)
}

#calling fn
CS_table = final_table(CS_wilcoxon_values)
CS_table
Ecig_table = final_table(Ecig_wilcoxon_values)
Ecig_table

,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,-0.53906101971814,182,9.97093472527582e-08,2.991280e-07
W1,NLF,ClusterB,0.0119333767804518,83,0.720314184153193,7.203142e-01
W2,NLF,ClusterC,-0.123670513346295,119,0.185184786347796,2.777772e-01
W3,NELF,ClusterA,0.00619092076850924,74,0.429599517406759,8.299838e-01
W4,NELF,ClusterB,-0.0136233084574024,96,0.829983797231071,8.299838e-01
W5,NELF,ClusterC,-0.0106803302321949,98,0.756376761739529,8.299838e-01
W6,Sputum,ClusterA,0.0391139960871419,89,0.942952393772154,9.429524e-01
W7,Sputum,ClusterB,0.519577398194044,0,9.97093472527582e-08,2.991280e-07
W8,Sputum,ClusterC,-0.0148431078782217,114,0.279954033990917,4.199311e-01


,Compartment,Cluster,AD,Stat,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,ClusterA,-0.0226870369614262,191,0.00346913507969652,5.203703e-03
W1,NLF,ClusterB,-0.493596987425873,238,7.54197509809517e-09,2.262593e-08
W2,NLF,ClusterC,-0.122510267739814,133,0.597070760978688,5.970708e-01
W3,NELF,ClusterA,-0.0202826984802277,167,0.0586119692464652,1.758359e-01
W4,NELF,ClusterB,0.0224998516702876,85,0.186378706515446,2.621173e-01
W5,NELF,ClusterC,0.0128519141367842,90,0.262117317119595,2.621173e-01
W6,Sputum,ClusterA,0.0246558747804312,126,0.799302563394779,7.993026e-01
W7,Sputum,ClusterB,0.0257790439966781,87,0.214531813512221,3.217977e-01
W8,Sputum,ClusterC,0.0193029611027928,84,0.173340434102888,3.217977e-01


In [7]:
#exporting tables
write.csv(CS_table, paste0(Output,"/", cur_date, "_CS_Distribution_Analysis.csv"), row.names = FALSE)
write.csv(Ecig_table, paste0(Output,"/", cur_date, "_Ecig_Distribution_Analysis.csv"), row.names = FALSE)